In [1]:
#Author : Muhammad Wafiq Kamaluddin --()-- Consiglere 2021


import paho.mqtt.client as mqtt

from PyQt5.QtWidgets import *
from PyQt5.QtCore import *
from PyQt5.QtGui import *

from PyQt5 import *
# from time import sleep
# from PIL import Image

import sys
import cv2
import os
import numpy as np
import os.path
import csv

help_text = "How to use this device \n 1. Librarian will chose book for blind person which book they will read \n 2. Next the librarian click the button to play audio based on character on the book. \n 3. The audio will play after OCR process to entire sheet of the book"

class VideoThread(QThread):
    
    change_pixmap_signal_camera = pyqtSignal(np.ndarray)
    change_pixmap_signal_microscope = pyqtSignal(np.ndarray)
    
    def __init__(self):
        super().__init__()
        self._run_flag = True


    def run(self):
        # capture from web cam
        cap_camera = cv2.VideoCapture('1.mkv')
        cap_microscope = cv2.VideoCapture('2.mkv')
#         cap_camera = cv2.VideoCapture(0)
#         cap_microscope = cv2.VideoCapture(1)
        while self._run_flag:
            # print(self.flagCapture)
            ret_camera, cv_img_camera = cap_camera.read()
            ret_microscope, cv_img_microscope = cap_microscope.read()
            if ret_camera :
                self.change_pixmap_signal_camera.emit(cv_img_camera)
            else:
                cap_camera = cv2.VideoCapture('1.mkv')
            if ret_microscope:       
                self.change_pixmap_signal_microscope.emit(cv_img_microscope)
            else:
                cap_microscope = cv2.VideoCapture('2.mkv')
        cap_camera.release()
        cap_microscope.release()

    def stop(self):
        """Sets run flag to False and waits for thread to finish"""
        self._run_flag = False
        self.wait()
        
class MQTT():
    
    def on_connect(mqttc, obj, flags, rc):
        print("rc: " + str(rc))


    def on_message(mqttc, obj, msg):
        print(msg.topic + " " + str(msg.qos) + " " + str(msg.payload))


    def on_publish(mqttc, obj, mid):
        print("mid: " + str(mid))


    def on_subscribe(mqttc, obj, mid, granted_qos):
        print("Subscribed: " + str(mid) + " " + str(granted_qos))


    def on_log(mqttc, obj, level, string):
        print(string)
        
    # mqttc = mqtt.Client()
    # mqttc.on_message = on_message
    # mqttc.on_connect = on_connect
    # mqttc.on_publish = on_publish
    # mqttc.on_subscribe = on_subscribe
    # # Uncomment to enable debug messages
    # # mqttc.on_log = on_log
    # mqttc.connect("broker.hivemq.com", 1883, 60)
    # mqttc.subscribe("/motor_monitoring/logger", 0)

    # mqttc.loop_forever()

class App(QWidget):
    def __init__(self):
        super().__init__()
        
        # variable
        self.imageMicNow = 0
        self.imageCamNow = 0
        self.flagCaptureCam = False
        self.flagCaptureMic = False
        self.flagSaveCam = False
        self.flagSaveMic = False
        self.iCam = 0
        self.iMic = 0
        
        self.max_cam_R = 0  
        self.max_cam_G = 0
        self.max_cam_B = 0
        
        self.max_cam_R = 0
        self.max_cam_G = 0
        self.max_cam_B = 0
        
        self.mean_mic_R = 0
        self.mean_mic_G = 0
        self.mean_mic_B = 0
        
        self.max_mic_R = 0
        self.max_mic_G = 0
        self.max_mic_B = 0

        self.begin = QtCore.QPoint()
        self.end = QtCore.QPoint()
        
        self.cap_cam_begin = QtCore.QPoint()
        self.cap_cam_end = QtCore.QPoint()
        self.cap_mic_begin = QtCore.QPoint()
        self.cap_mic_end = QtCore.QPoint()

        self.path_file = 'data/data.csv'
        
        # layout setting
        
        self.screen_resolution = app.desktop().screenGeometry()
        self.MAX_WIDTH, self.MAX_HEIGH = self.screen_resolution.width(), self.screen_resolution.height()
        
        self.resize(self.MAX_WIDTH,self.MAX_HEIGH)
        self.setWindowTitle('Aplikasi Monitoring Air Tambak')
        
        self.grid_left_x_init = int((self.MAX_WIDTH-100)/2-50) 
        self.grid_left_y_init = int(20.0)
        self.grid_left_x_end = int((self.MAX_WIDTH-100)/2-50)
        self.grid_left_y_end = int(self.MAX_HEIGH-40.0)
        
        self.grid_right_x_init = int((self.MAX_WIDTH-100)/2+200) 
        self.grid_right_y_init = int(20.0)
        self.grid_right_x_end = int((self.MAX_WIDTH-100)/2+200)
        self.grid_right_y_end = int(self.MAX_HEIGH-40.0)
        
        self.icon_size_x = 50
        self.icon_size_y = 50
        
        self.icon_1_left_x = int(self.grid_left_x_init + (self.grid_right_x_init-self.grid_left_x_end-30)/2 - 70)
        self.icon_1_left_y = int(self.grid_left_y_init+50)
        self.icon_1_left_caption_x = int(self.icon_1_left_x + self.icon_size_x/2 -10) - 40
        self.icon_1_left_caption_y = int(self.icon_1_left_y + self.icon_size_y + 20)
        
        self.icon_2_left_x = self.icon_1_left_x
        self.icon_2_left_y = self.icon_1_left_y + 150
        self.icon_2_left_caption_x = self.icon_1_left_caption_x
        self.icon_2_left_caption_y = self.icon_1_left_caption_y + 150
        
        self.icon_3_left_x = self.icon_2_left_x 
        self.icon_3_left_y = self.icon_2_left_y + 150
        self.icon_3_left_caption_x = self.icon_2_left_caption_x + 20
        self.icon_3_left_caption_y = self.icon_2_left_caption_y + 150
        
        self.icon_4_left_x = self.icon_3_left_x
        self.icon_4_left_y = self.icon_3_left_y + 150
        self.icon_4_left_caption_x = self.icon_3_left_caption_x
        self.icon_4_left_caption_y = self.icon_3_left_caption_y + 150
        
        self.icon_1_right_x = int(self.grid_right_x_init - (self.grid_right_x_init-self.grid_left_x_end-30)/2) + 10
        self.icon_1_right_y = int(self.grid_right_y_init+50)
        self.icon_1_right_caption_x = int(self.icon_1_right_x + self.icon_size_x/2 - 30) 
        self.icon_1_right_caption_y = int(self.icon_1_right_y + self.icon_size_y + 20)

        self.icon_2_right_x = self.icon_1_right_x
        self.icon_2_right_y = self.icon_1_right_y + 150
        self.icon_2_right_caption_x = self.icon_1_right_caption_x - 10
        self.icon_2_right_caption_y = self.icon_1_right_caption_y + 150

        self.icon_3_right_x = self.icon_2_right_x
        self.icon_3_right_y = self.icon_2_right_y + 150
        self.icon_3_right_caption_x = self.icon_2_right_caption_x + 20
        self.icon_3_right_caption_y = self.icon_2_right_caption_y + 150

        self.icon_4_right_x = self.icon_3_right_x
        self.icon_4_right_y = self.icon_3_right_y + 150
        self.icon_4_right_caption_x = self.icon_3_right_caption_x - 10
        self.icon_4_right_caption_y = self.icon_3_right_caption_y + 150
        
        self.icon_5_x = self.icon_4_right_x - 50
        self.icon_5_y = self.icon_4_right_y + 100
        self.icon_5_caption_x = self.icon_4_right_caption_x -50
        self.icon_5_caption_y = self.icon_4_right_caption_y + 100
        
        self.textbox_x = 50
        self.textbox_y = 20
        # mean
        self.label_camera_mean_rgb_R_x = self.grid_right_x_init + 40
        self.label_camera_mean_rgb_R_y = self.grid_right_y_init + 80
        self.data_camera_mean_rgb_R_x = self.label_camera_mean_rgb_R_x + 25
        self.data_camera_mean_rgb_R_y = self.label_camera_mean_rgb_R_y - 3
        
        self.label_camera_mean_rgb_G_x = self.grid_right_x_init + 40
        self.label_camera_mean_rgb_G_y = self.grid_right_y_init + 120
        self.data_camera_mean_rgb_G_x = self.label_camera_mean_rgb_G_x + 25
        self.data_camera_mean_rgb_G_y = self.label_camera_mean_rgb_G_y - 3
        
        self.label_camera_mean_rgb_B_x = self.grid_right_x_init + 40
        self.label_camera_mean_rgb_B_y = self.grid_right_y_init + 160
        self.data_camera_mean_rgb_B_x = self.label_camera_mean_rgb_B_x + 25
        self.data_camera_mean_rgb_B_y = self.label_camera_mean_rgb_B_y - 3
        
        # max
        self.label_camera_max_rgb_R_x = self.grid_right_x_init + 170
        self.label_camera_max_rgb_R_y = self.grid_right_y_init + 80
        self.data_camera_max_rgb_R_x = self.label_camera_max_rgb_R_x + 25
        self.data_camera_max_rgb_R_y = self.label_camera_max_rgb_R_y - 3
        
        self.label_camera_max_rgb_G_x = self.grid_right_x_init + 170
        self.label_camera_max_rgb_G_y = self.grid_right_y_init + 120
        self.data_camera_max_rgb_G_x = self.label_camera_max_rgb_G_x + 25
        self.data_camera_max_rgb_G_y = self.label_camera_max_rgb_G_y - 3
        
        self.label_camera_max_rgb_B_x = self.grid_right_x_init + 170
        self.label_camera_max_rgb_B_y = self.grid_right_y_init + 160
        self.data_camera_max_rgb_B_x = self.label_camera_max_rgb_B_x + 25
        self.data_camera_max_rgb_B_y = self.label_camera_max_rgb_B_y - 3
        
        
        self.label_microscope_mean_rgb_R_x = self.grid_right_x_init + 290
        self.label_microscope_mean_rgb_R_y = self.grid_right_y_init + 80
        self.data_microscope_mean_rgb_R_x = self.label_microscope_mean_rgb_R_x + 25
        self.data_microscope_mean_rgb_R_y = self.label_microscope_mean_rgb_R_y - 3
        
        self.label_microscope_mean_rgb_G_x = self.grid_right_x_init + 290
        self.label_microscope_mean_rgb_G_y = self.grid_right_y_init + 120
        self.data_microscope_mean_rgb_G_x = self.label_microscope_mean_rgb_G_x + 25
        self.data_microscope_mean_rgb_G_y = self.label_microscope_mean_rgb_G_y - 3
        
        self.label_microscope_mean_rgb_B_x = self.grid_right_x_init + 290
        self.label_microscope_mean_rgb_B_y = self.grid_right_y_init + 160
        self.data_microscope_mean_rgb_B_x = self.label_microscope_mean_rgb_B_x + 25
        self.data_microscope_mean_rgb_B_y = self.label_microscope_mean_rgb_B_y - 3
        
        # max
        self.label_microscope_max_rgb_R_x = self.grid_right_x_init + 420
        self.label_microscope_max_rgb_R_y = self.grid_right_y_init + 80
        self.data_microscope_max_rgb_R_x = self.label_microscope_max_rgb_R_x + 25
        self.data_microscope_max_rgb_R_y = self.label_microscope_max_rgb_R_y - 3
        
        self.label_microscope_max_rgb_G_x = self.grid_right_x_init + 420
        self.label_microscope_max_rgb_G_y = self.grid_right_y_init + 120
        self.data_microscope_max_rgb_G_x = self.label_microscope_max_rgb_G_x + 25
        self.data_microscope_max_rgb_G_y = self.label_microscope_max_rgb_G_y - 3
        
        self.label_microscope_max_rgb_B_x = self.grid_right_x_init + 420
        self.label_microscope_max_rgb_B_y = self.grid_right_y_init + 160
        self.data_microscope_max_rgb_B_x = self.label_microscope_max_rgb_B_x + 25
        self.data_microscope_max_rgb_B_y = self.label_microscope_max_rgb_B_y - 3
        
    #     font
        self.font_title = QtGui.QFont('Arial', 14)
        self.font_sub_title = QtGui.QFont('Arial', 10)
        self.font_data_label = QtGui.QFont('Arial', 10)

    #     title
        self.title_left = QLabel(self)
        self.title_left.setText("Tampilan")
        self.title_left.setFont(self.font_title)
        self.title_left.move(50, 10)
        
#         subtitle
        self.label = QLabel(self)
        self.label.setText("Kamera")
        self.label.setFont(self.font_sub_title)
        self.label.move(50, 55)
        
        self.label = QLabel(self)
        self.label.setText("Mikroskop")
        self.label.setFont(self.font_sub_title)
        self.label.move(50, 390)
        
#         Controller
        self.title_center = QLabel(self)
        self.title_center.setText("Kontroler")
        self.title_center.setFont(self.font_title)
        self.title_center.move(590, 10)
#         1
        self.btn_camera = QPushButton(self)
        self.btn_camera.setIcon(QtGui.QIcon('assets/icon/camera.png'))
        self.btn_camera.setIconSize(QtCore.QSize(self.icon_size_x,self.icon_size_y))
        self.btn_camera.move(self.icon_1_left_x,self.icon_1_left_y)
        self.btn_camera.show()
        self.btn_camera.clicked.connect(self.captureCamEvent)
        self.btn_camera_label = QLabel(self)
        self.btn_camera_label.setText("ambil gambar \n  kamera")
        self.btn_camera_label.setFont(self.font_sub_title)
        self.btn_camera_label.move(self.icon_1_left_caption_x, self.icon_1_left_caption_y)
        
        self.btn_mic = QPushButton(self)
        self.btn_mic.setIcon(QtGui.QIcon('assets/icon/camera.png'))
        self.btn_mic.setIconSize(QtCore.QSize(self.icon_size_x,self.icon_size_y))
        self.btn_mic.move(self.icon_1_right_x,self.icon_1_right_y)
        self.btn_mic.show()
        self.btn_mic.clicked.connect(self.captureMicEvent)
        self.btn_mic_label = QLabel(self)
        self.btn_mic_label.setText("ambil gambar \n  mikroskop")
        self.btn_mic_label.setFont(self.font_sub_title)
        self.btn_mic_label.move(self.icon_1_right_caption_x, self.icon_1_right_caption_y)
        
#         2
        self.btn_camera_save_to_csv = QPushButton(self)
        self.btn_camera_save_to_csv.setIcon(QtGui.QIcon('assets/icon/camera.png'))
        self.btn_camera_save_to_csv.setIconSize(QtCore.QSize(self.icon_size_x,self.icon_size_y))
        self.btn_camera_save_to_csv.move(self.icon_2_left_x,self.icon_2_left_y)
        self.btn_camera_save_to_csv.show()
        self.btn_camera_save_to_csv.clicked.connect(self.getParamCam)
        self.btn_camera_label_save_to_csv = QLabel(self)
        self.btn_camera_label_save_to_csv.setText("simpan ke csv")
        self.btn_camera_label_save_to_csv.setFont(self.font_sub_title)
        self.btn_camera_label_save_to_csv.move(self.icon_2_left_caption_x, self.icon_2_left_caption_y)
        
        # self.btn_microscope_save_to_csv = QPushButton(self)
        # self.btn_microscope_save_to_csv.setIcon(QtGui.QIcon('assets/icon/camera.png'))
        # self.btn_microscope_save_to_csv.setIconSize(QtCore.QSize(self.icon_size_x,self.icon_size_y))
        # self.btn_microscope_save_to_csv.move(self.icon_2_right_x,self.icon_2_right_y)
        # self.btn_microscope_save_to_csv.show()
        # self.btn_microscope_save_to_csv.clicked.connect(self.getParamMic)
        # self.textbox.setReadOnly(True)
        # self.btn_microscope_save_to_csv_label = QLabel(self)
        # self.btn_microscope_save_to_csv_label.setText("analisa dan simpan \n ke csv (mikroskop)")
        # self.btn_microscope_save_to_csv_label.setFont(self.font_sub_title)
        # self.btn_microscope_save_to_csv_label.move(self.icon_2_right_caption_x, self.icon_2_right_caption_y)
        self.max_B = QLabel(self)
        self.max_B.setText("Kelas  = ")
        self.max_B.setFont(self.font_data_label)
        self.max_B.move(self.icon_2_right_x - 30,self.icon_2_right_y + 20)
        self.textbox_class = QLineEdit(self)
        self.textbox_class.move(self.icon_2_right_x + 50,self.icon_2_right_y + 20)
        self.textbox_class.resize(self.textbox_x ,self.textbox_y)
        self.textbox_class.setText('null')
        
        
        self.max_B = QLabel(self)
        self.max_B.setText("Konsenstrasi  = ")
        self.max_B.setFont(self.font_data_label)
        self.max_B.move(self.icon_2_right_x - 30,self.icon_2_right_y + 40)
        self.textbox_concentration = QLineEdit(self)
        self.textbox_concentration.move(self.icon_2_right_x + 50,self.icon_2_right_y + 40)
        self.textbox_concentration.resize(self.textbox_x,self.textbox_y)
        self.textbox_concentration.setText('null')
        
#         3
        self.btn_classify_train = QPushButton(self)
        self.btn_classify_train.setIcon(QtGui.QIcon('assets/icon/camera.png'))
        self.btn_classify_train.setIconSize(QtCore.QSize(self.icon_size_x,self.icon_size_y))
        self.btn_classify_train.move(self.icon_3_left_x,self.icon_3_left_y)
        self.btn_classify_train.show()
        self.btn_classify_train.clicked.connect(self.help_window)
        self.btn_classify_label_train = QLabel(self)
        self.btn_classify_label_train.setText("train model \n klasifikasi")
        self.btn_classify_label_train.setFont(self.font_sub_title)
        self.btn_classify_label_train.move(self.icon_3_left_caption_x, self.icon_3_left_caption_y)
        
        self.btn_regresion_train = QPushButton(self)
        self.btn_regresion_train.setIcon(QtGui.QIcon('assets/icon/camera.png'))
        self.btn_regresion_train.setIconSize(QtCore.QSize(self.icon_size_x,self.icon_size_y))
        self.btn_regresion_train.move(self.icon_3_right_x,self.icon_3_right_y)
        self.btn_regresion_train.show()
        self.btn_regresion_train.clicked.connect(self.help_window)
        self.btn_regresion_label_train = QLabel(self)
        self.btn_regresion_label_train.setText("train model \n    regresi")
        self.btn_regresion_label_train.setFont(self.font_sub_title)
        self.btn_regresion_label_train.move(self.icon_3_right_caption_x, self.icon_3_right_caption_y)
        
#         4
        self.btn_class_predict = QPushButton(self)
        self.btn_class_predict.setIcon(QtGui.QIcon('assets/icon/camera.png'))
        self.btn_class_predict.setIconSize(QtCore.QSize(self.icon_size_x,self.icon_size_y))
        self.btn_class_predict.move(self.icon_4_left_x,self.icon_4_left_y)
        self.btn_class_predict.show()
        self.btn_class_predict.clicked.connect(self.help_window)
        self.btn_class_predict_label = QLabel(self)
        self.btn_class_predict_label.setText("prediksi kelas")
        self.btn_class_predict_label.setFont(self.font_sub_title)
        self.btn_class_predict_label.move(self.icon_4_left_caption_x, self.icon_4_left_caption_y)
        
        self.btn_consentration_predict = QPushButton(self)
        self.btn_consentration_predict.setIcon(QtGui.QIcon('assets/icon/camera.png'))
        self.btn_consentration_predict.setIconSize(QtCore.QSize(self.icon_size_x,self.icon_size_y))
        self.btn_consentration_predict.move(self.icon_4_right_x,self.icon_4_right_y)
        self.btn_consentration_predict.show()
        self.btn_consentration_predict.clicked.connect(self.help_window)
        self.btn_consentration_predict_label = QLabel(self)
        self.btn_consentration_predict_label.setText("prediksi konsenstrasi")
        self.btn_consentration_predict_label.setFont(self.font_sub_title)
        self.btn_consentration_predict_label.move(self.icon_4_right_caption_x, self.icon_4_right_caption_y)
        
        self.btn_next_data = QPushButton(self)
        self.btn_next_data.setIcon(QtGui.QIcon('assets/icon/camera.png'))
        self.btn_next_data.setIconSize(QtCore.QSize(self.icon_size_x,self.icon_size_y))
        self.btn_next_data.move(self.icon_5_x,self.icon_5_y)
        self.btn_next_data.show()
        self.btn_next_data.clicked.connect(self.nextDataEvent)
        self.btn_next_data_label = QLabel(self)
        self.btn_next_data_label.setText("data selanjutnya")
        self.btn_next_data_label.setFont(self.font_sub_title)
        self.btn_next_data_label.move(self.icon_5_caption_x, self.icon_5_caption_y)    
        
#     kamera
        self.display_width = 480
        self.display_height = 320
        # create the label that holds the image
        self.image_label_camera = QLabel(self)
        self.image_label_camera.setText("Kamera")
        self.image_label_camera.resize(self.display_width, self.display_height)
        self.image_label_camera.move(50, 60)
        
        self.thread_camera = VideoThread()
        # connect its signal to the update_image slot
        self.thread_camera.change_pixmap_signal_camera.connect(self.update_image_camera)
        # self.image_label_camera.raise_()
               
#         Mikroskop
        self.display_width = 480
        self.display_height = 320
        # create the label that holds the image
        self.image_label_microscope = QLabel(self)
        self.image_label_microscope.setText("Mikroskop")
        self.image_label_microscope.resize(self.display_width, self.display_height)
        self.image_label_microscope.move(50, 400)
        
        self.thread_camera.change_pixmap_signal_microscope.connect(self.update_image_microscope)
        # self.image_label_microscope.raise_()
        # self.thread_camera.lower()
        
        # start the thread
        self.thread_camera.start()
#         divider
        
        self.paintEvent(self)

#         data
        self.title_center = QLabel(self)
        self.title_center.setText("Data")
        self.title_center.setFont(self.font_title)
        self.title_center.move(850, 10)
        
        # camera
        self.label_camera = QLabel(self)
        self.label_camera.setText("Camera")
        self.label_camera.setFont(self.font_sub_title)
        self.label_camera.move(870, 40)
        
        self.label_mean_rgb = QLabel(self)
        self.label_mean_rgb.setText("Mean RGB")
        self.label_mean_rgb.setFont(self.font_sub_title)
        self.label_mean_rgb.move(870, 70)
        
        self.mean_R_camera = QLabel(self)
        self.mean_R_camera.setText("R = ")
        self.mean_R_camera.setFont(self.font_data_label)
        self.mean_R_camera.move(self.label_camera_mean_rgb_R_x, self.label_camera_mean_rgb_R_y)
        # self.mean_R_camera.lower()
        self.textbox_mean_R_cam = QLineEdit(self)
        self.textbox_mean_R_cam.move(self.data_camera_mean_rgb_R_x, self.data_camera_mean_rgb_R_y)
        self.textbox_mean_R_cam.resize(self.textbox_x,self.textbox_y)
        self.textbox_mean_R_cam.setReadOnly(True)
        self.textbox_mean_R_cam.setText('kenek')
        # self.textbox_mean_R_cam.lower()
        
        self.mean_G_camera = QLabel(self)
        self.mean_G_camera.setText("G = ")
        self.mean_G_camera.setFont(self.font_data_label)
        self.mean_G_camera.move(self.label_camera_mean_rgb_G_x, self.label_camera_mean_rgb_G_y)
        self.textbox_mean_G_cam = QLineEdit(self)
        self.textbox_mean_G_cam.move(self.data_camera_mean_rgb_G_x, self.data_camera_mean_rgb_G_y)
        self.textbox_mean_G_cam.resize(self.textbox_x,self.textbox_y)
        self.textbox_mean_G_cam.setReadOnly(True)
        self.textbox_mean_G_cam.setText('kenek')
        
        self.mean_B_camera = QLabel(self)
        self.mean_B_camera.setText("B = ")
        self.mean_B_camera.setFont(self.font_data_label)
        self.mean_B_camera.move(self.label_camera_mean_rgb_B_x, self.label_camera_mean_rgb_B_y)
        self.textbox_mean_B_cam = QLineEdit(self)
        self.textbox_mean_B_cam.move(self.data_camera_mean_rgb_B_x, self.data_camera_mean_rgb_B_y)
        self.textbox_mean_B_cam.resize(self.textbox_x,self.textbox_y)
        self.textbox_mean_B_cam.setReadOnly(True)
        self.textbox_mean_B_cam.setText('kenek')
        
        self.label_max_rgb = QLabel(self)
        self.label_max_rgb.setText("max RGB")
        self.label_max_rgb.setFont(self.font_sub_title)
        self.label_max_rgb.move(1000, 70)
        
        self.max_R_camera = QLabel(self)
        self.max_R_camera.setText("R = ")
        self.max_R_camera.setFont(self.font_data_label)
        self.max_R_camera.move(self.label_camera_max_rgb_R_x, self.label_camera_max_rgb_R_y)
        self.textbox_max_R_cam = QLineEdit(self)
        self.textbox_max_R_cam.move(self.data_camera_max_rgb_R_x, self.data_camera_max_rgb_R_y)
        self.textbox_max_R_cam.resize(self.textbox_x,self.textbox_y)
        self.textbox_max_R_cam.setReadOnly(True)
        self.textbox_max_R_cam.setText('kenek')
        
        self.max_G_camera = QLabel(self)
        self.max_G_camera.setText("G = ")
        self.max_G_camera.setFont(self.font_data_label)
        self.max_G_camera.move(self.label_camera_max_rgb_G_x, self.label_camera_max_rgb_G_y)
        self.textbox_max_G_cam = QLineEdit(self)
        self.textbox_max_G_cam.move(self.data_camera_max_rgb_G_x, self.data_camera_max_rgb_G_y)
        self.textbox_max_G_cam.resize(self.textbox_x,self.textbox_y)
        self.textbox_max_G_cam.setReadOnly(True)
        self.textbox_max_G_cam.setText('kenek')
        
        self.max_B_camera = QLabel(self)
        self.max_B_camera.setText("B = ")
        self.max_B_camera.setFont(self.font_data_label)
        self.max_B_camera.move(self.label_camera_max_rgb_B_x, self.label_camera_max_rgb_B_y)
        self.textbox_max_B_cam = QLineEdit(self)
        self.textbox_max_B_cam.move(self.data_camera_max_rgb_B_x, self.data_camera_max_rgb_B_y)
        self.textbox_max_B_cam.resize(self.textbox_x,self.textbox_y)
        self.textbox_max_B_cam.setReadOnly(True)
        self.textbox_max_B_cam.setText('kenek')
        
        # microscope
        self.label_microscope = QLabel(self)
        self.label_microscope.setText("Microscope")
        self.label_microscope.setFont(self.font_sub_title)
        self.label_microscope.move(1130, 40)
        
        self.label_microscope = QLabel(self)
        self.label_microscope.setText("Mean RGB")
        self.label_microscope.setFont(self.font_sub_title)
        self.label_microscope.move(1130, 70)
        
        self.mean_R_microscope = QLabel(self)
        self.mean_R_microscope.setText("R = ")
        self.mean_R_microscope.setFont(self.font_data_label)
        self.mean_R_microscope.move(self.label_microscope_mean_rgb_R_x, self.label_microscope_mean_rgb_R_y)
        self.textbox_mean_R_mic = QLineEdit(self)
        self.textbox_mean_R_mic.move(self.data_microscope_mean_rgb_R_x, self.data_microscope_mean_rgb_R_y)
        self.textbox_mean_R_mic.resize(self.textbox_x,self.textbox_y)
        self.textbox_mean_R_mic.setReadOnly(True)
        self.textbox_mean_R_mic.setText('kenek')
        
        self.mean_G_microscope = QLabel(self)
        self.mean_G_microscope.setText("G = ")
        self.mean_G_microscope.setFont(self.font_data_label)
        self.mean_G_microscope.move(self.label_microscope_mean_rgb_G_x, self.label_microscope_mean_rgb_G_y)
        self.textbox_mean_G_mic = QLineEdit(self)
        self.textbox_mean_G_mic.move(self.data_microscope_mean_rgb_G_x, self.data_microscope_mean_rgb_G_y)
        self.textbox_mean_G_mic.resize(self.textbox_x,self.textbox_y)
        self.textbox_mean_G_mic.setReadOnly(True)
        self.textbox_mean_G_mic.setText('kenek')
        
        self.mean_B_microscope = QLabel(self)
        self.mean_B_microscope.setText("B = ")
        self.mean_B_microscope.setFont(self.font_data_label)
        self.mean_B_microscope.move(self.label_microscope_mean_rgb_B_x, self.label_microscope_mean_rgb_B_y)
        self.textbox_mean_B_mic = QLineEdit(self)
        self.textbox_mean_B_mic.move(self.data_microscope_mean_rgb_B_x, self.data_microscope_mean_rgb_B_y)
        self.textbox_mean_B_mic.resize(self.textbox_x,self.textbox_y)
        self.textbox_mean_B_mic.setReadOnly(True)
        self.textbox_mean_B_mic.setText('kenek')
        
        self.label_microscope = QLabel(self)
        self.label_microscope.setText("max RGB")
        self.label_microscope.setFont(self.font_sub_title)
        self.label_microscope.move(1250, 70)
        
        self.max_R_microscope = QLabel(self)
        self.max_R_microscope.setText("R = ")
        self.max_R_microscope.setFont(self.font_data_label)
        self.max_R_microscope.move(self.label_microscope_max_rgb_R_x, self.label_microscope_max_rgb_R_y)
        self.textbox_max_R_mic = QLineEdit(self)
        self.textbox_max_R_mic.move(self.data_microscope_max_rgb_R_x, self.data_microscope_max_rgb_R_y)
        self.textbox_max_R_mic.resize(self.textbox_x,self.textbox_y)
        self.textbox_max_R_mic.setReadOnly(True)
        self.textbox_max_R_mic.setText('kenek')
        
        self.max_G_microscope = QLabel(self)
        self.max_G_microscope.setText("G = ")
        self.max_G_microscope.setFont(self.font_data_label)
        self.max_G_microscope.move(self.label_microscope_max_rgb_G_x, self.label_microscope_max_rgb_G_y)
        self.textbox_max_G_mic = QLineEdit(self)
        self.textbox_max_G_mic.move(self.data_microscope_max_rgb_G_x, self.data_microscope_max_rgb_G_y)
        self.textbox_max_G_mic.resize(self.textbox_x,self.textbox_y)
        self.textbox_max_G_mic.setReadOnly(True)
        self.textbox_max_G_mic.setText('kenek')
        
        self.max_B_microscope = QLabel(self)
        self.max_B_microscope.setText("B = ")
        self.max_B_microscope.setFont(self.font_data_label)
        self.max_B_microscope.move(self.label_microscope_max_rgb_B_x, self.label_microscope_max_rgb_B_y)
        self.textbox_max_B_mic = QLineEdit(self)
        self.textbox_max_B_mic.move(self.data_microscope_max_rgb_B_x, self.data_microscope_max_rgb_B_y)
        self.textbox_max_B_mic.resize(self.textbox_x,self.textbox_y)
        self.textbox_max_B_mic.setReadOnly(True)
        self.textbox_max_B_mic.setText('kenek')
        
        self.label = QLabel(self)
        self.label.setText("Hasil Training")
        self.label.setFont(self.font_sub_title)
        self.label.move(1065, 230)
        
        self.label = QLabel(self)
        self.label.setText("Klasifikasi")
        self.label.setFont(self.font_sub_title)
        self.label.move(1000, 280)
        
        self.max_B = QLabel(self)
        self.max_B.setText("Akurasi Model = ")
        self.max_B.setFont(self.font_data_label)
        self.max_B.move(900, 320)
        self.textbox = QLineEdit(self)
        self.textbox.move(1000, 320)
        self.textbox.resize(self.textbox_x,self.textbox_y)
        self.textbox.setReadOnly(True)
        self.textbox.setText('kenek')
        
        self.label = QLabel(self)
        self.label.setText("Regresi")
        self.label.setFont(self.font_sub_title)
        self.label.move(1160, 280)
        
        self.max_B = QLabel(self)
        self.max_B.setText("RMSE = ")
        self.max_B.setFont(self.font_data_label)
        self.max_B.move(1150, 320)
        self.textbox = QLineEdit(self)
        self.textbox.move(1200, 320)
        self.textbox.resize(self.textbox_x,self.textbox_y)
        self.textbox.setReadOnly(True)
        self.textbox.setText('kenek')
        
        self.max_B = QLabel(self)
        self.max_B.setText("MSE  = ")
        self.max_B.setFont(self.font_data_label)
        self.max_B.move(1150, 350)
        self.textbox = QLineEdit(self)
        self.textbox.move(1200, 350)
        self.textbox.resize(self.textbox_x,self.textbox_y)
        self.textbox.setReadOnly(True)
        self.textbox.setText('kenek')
        
        self.label = QLabel(self)
        self.label.setText("Hasil Prediksi")
        self.label.setFont(self.font_sub_title)
        self.label.move(1065, 410)
        
        self.label = QLabel(self)
        self.label.setText("Klasifikasi")
        self.label.setFont(self.font_sub_title)
        self.label.move(1000, 440)
        
        self.max_B = QLabel(self)
        self.max_B.setText("Kelas = ")
        self.max_B.setFont(self.font_data_label)
        self.max_B.move(900, 480)
        self.textbox = QLineEdit(self)
        self.textbox.move(1000, 480)
        self.textbox.resize(self.textbox_x,self.textbox_y)
        self.textbox.setReadOnly(True)
        self.textbox.setText('kenek')
        
        self.label = QLabel(self)
        self.label.setText("Regresi")
        self.label.setFont(self.font_sub_title)
        self.label.move(1160, 440)
        
        self.max_B = QLabel(self)
        self.max_B.setText("Konsenstrasi = ")
        self.max_B.setFont(self.font_data_label)
        self.max_B.move(1150, 480)
        self.textbox = QLineEdit(self)
        self.textbox.move(1240, 480)
        self.textbox.resize(self.textbox_x,self.textbox_y)
        self.textbox.setReadOnly(True)
        self.textbox.setText('kenek')
        
        
    def closeEvent(self, event):
        self.thread_camera.stop()
        event.accept()
        
    def paintEvent(self, event):
        painter = QPainter(self)
        painter.setPen(Qt.black)
        
        # painter.drawLine(self.grid_left_x_init, self.grid_left_y_init, self.grid_left_x_end, self.grid_left_y_end)
        # painter.drawLine(self.grid_right_x_init, self.grid_right_y_init, self.grid_right_x_end, self.grid_right_y_end)
         
        qp = QtGui.QPainter(self)
        br = QtGui.QBrush(QtGui.QColor(100, 10, 10, 40)) 
        qp.setBrush(br)   
        qp.drawRect(QtCore.QRect(self.begin, self.end))
        
    def mousePressEvent(self, event):
        # self.thread_camera.stop()
        self.begin = event.pos()
        # self.end = event.pos()
        self.update()

    def mouseMoveEvent(self, event):
        self.end = event.pos()
        self.update()
        # pass

    def mouseReleaseEvent(self, event):
        # self.thread_camera.start()
        # self.begin = event.pos()
        self.end = event.pos()
        self.update()
        
#         painter.setPen(Qt.blue)
#         painter.drawRect(120,10,80,80)

#         rectf = QRectF(230.0,10.0,80.0,80.0)
#         painter.drawRoundedRect(rectf,20,20)

#         p1 = [QPoint(10,100),QPoint(220,110),QPoint(220,190)]
#         painter.drawPolyline(QPolygon(p1))

#         p2 = [QPoint(120,110),QPoint(220,110),QPoint(220,190)]
#         painter.drawPolygon(QPolygon(p2))
    def help_window(self,event):
        mbox = QMessageBox()

        mbox.setText(help_text)
        # mbox.setDetailedText("You are now a disciple and subject of the all-knowing Guru")
        mbox.setStandardButtons(QMessageBox.Ok | QMessageBox.Cancel)

        mbox.exec_()
        
    def captureMicEvent(self,event):
        self.flagCaptureMic = True
        self.cap_mic_begin = self.begin
        self.cap_mic_end = self.end
        # print(self.flagCaptureCam)
    
    def captureCamEvent(self,event):
        self.flagCaptureCam = True
        self.cap_cam_begin = self.begin
        self.cap_cam_end = self.end
        # print(self.flagCaptureMic)
        
    def nextDataEvent(self,event):
        self.flagCaptureMic = False
        self.flagCaptureCam = False
        self.flagSaveCam = False
        self.flagSaveMic = False
    
    
        
    def getParamCam(self,event):
        
        # self.thread_camera.stop()
        
        frame = cv2.imread('image/Camera/cam'+str(self.iCam-1)+'.jpg')
        
        
        height_final_begin = int((self.cap_cam_begin.y() - 400)/320 * frame.shape[0])
        width_final_begin = int((self.cap_cam_begin.x() - 50)/480 * frame.shape[1])

        height_final_end = int((self.cap_cam_end.y() - 350)/320 * frame.shape[0])
        width_final_end = int((self.cap_cam_end.x() - 50)/480 * frame.shape[1])

        if width_final_end > frame.shape[1]:
            width_final_end = frame.shape[1]
        if height_final_end > frame.shape[0]:
            height_final_end = frame.shape[0]

        frame = frame[height_final_begin:height_final_end,width_final_begin : width_final_end]
        
        cv2.imshow('frame cam',frame)
        
        self.imageCamNow = frame
                 
        histR=cv2.calcHist([frame], [0], None, [256], [0, 256])
        histG=cv2.calcHist([frame], [1], None, [256], [0, 256])
        histB=cv2.calcHist([frame], [2], None, [256], [0, 256])
        
        sumAllR=0
        sumMulR=0
        sumAllG=0
        sumMulG=0
        sumAllB=0
        sumMulB=0
             
        for i in range(0,256):
            if(histR[i]==np.max(histR)):
                self.max_cam_R = i 
                # self.inEMaR.set(i)
            sumAllR+=histR[i]
            sumMulR+=i*histR[i]
            if(histG[i]==np.max(histG)):
                self.max_cam_G = i 
                # self.inEMaG.set(i)
            sumAllG+=histG[i]
            sumMulG+=i*histG[i]
            if(histB[i]==np.max(histB)):
                self.max_cam_B = i 
                # self.inEMaB.set(i)
            sumAllB+=histB[i]
            sumMulB+=i*histB[i]      
        
        self.mean_cam_R = int(sumMulR/sumAllR)
        self.mean_cam_G = int(sumMulG/sumAllG)
        self.mean_cam_B = int(sumMulR/sumAllB)
        
        self.textbox_max_R_cam.setText(str(self.max_cam_R))
        self.textbox_max_G_cam.setText(str(self.max_cam_G))
        self.textbox_max_B_cam.setText(str(self.max_cam_B))
        
        self.textbox_mean_R_cam.setText(str(self.mean_cam_R))
        self.textbox_mean_G_cam.setText(str(self.mean_cam_G))
        self.textbox_mean_B_cam.setText(str(self.mean_cam_B))
        
        frame = cv2.imread('image/Microscope/mic'+str(self.iMic-1)+'.jpg')
        
        height_final_begin = int((self.cap_mic_begin.y() - 400)/320 * frame.shape[0])
        width_final_begin = int((self.cap_mic_begin.x() - 50)/480 * frame.shape[1])
        
        height_final_end = int((self.cap_mic_end.y() - 350)/320 * frame.shape[0])
        width_final_end = int((self.cap_mic_end.x() - 50)/480 * frame.shape[1])
        
        if width_final_end > frame.shape[1]:
            width_final_end = frame.shape[1]
        if height_final_end > frame.shape[0]:
            height_final_end = frame.shape[0]
            
        frame = frame[height_final_begin:height_final_end,width_final_begin : width_final_end]
        
        cv2.imshow('frame mic',frame)
        
        self.imageMicNow = frame
                 
        histR=cv2.calcHist([frame], [0], None, [256], [0, 256])
        histG=cv2.calcHist([frame], [1], None, [256], [0, 256])
        histB=cv2.calcHist([frame], [2], None, [256], [0, 256])
        
        sumAllR=0
        sumMulR=0
        sumAllG=0
        sumMulG=0
        sumAllB=0
        sumMulB=0
             
        for i in range(0,256):
            if(histR[i]==np.max(histR)):
                self.max_mic_R = i 
                # self.inEMaR.set(i)
            sumAllR+=histR[i]
            sumMulR+=i*histR[i]
            if(histG[i]==np.max(histG)):
                self.max_mic_G = i 
                # self.inEMaG.set(i)
            sumAllG+=histG[i]
            sumMulG+=i*histG[i]
            if(histB[i]==np.max(histB)):
                self.max_mic_B = i 
                # self.inEMaB.set(i)
            sumAllB+=histB[i]
            sumMulB+=i*histB[i]      
        
        self.mean_mic_R = int(sumMulR/sumAllR)
        self.mean_mic_G = int(sumMulG/sumAllG)
        self.mean_mic_B = int(sumMulR/sumAllB)
        
        self.textbox_max_R_mic.setText(str(self.max_mic_R))
        self.textbox_max_G_mic.setText(str(self.max_mic_G))
        self.textbox_max_B_mic.setText(str(self.max_mic_B))
        
        self.textbox_mean_R_mic.setText(str(self.mean_mic_R))
        self.textbox_mean_G_mic.setText(str(self.mean_mic_G))
        self.textbox_mean_B_mic.setText(str(self.mean_mic_B))
        
        class_data = self.textbox_class.text()
        concentration_data = self.textbox_concentration.text()
        if os.path.exists(self.path_file):
            row_list = [[self.mean_cam_R,self.mean_cam_G,self.mean_cam_B,self.max_cam_R,self.max_cam_G,self.max_cam_B,self.mean_mic_R,self.mean_mic_G,self.mean_mic_B,self.max_mic_R,self.max_mic_G,self.max_mic_B, class_data, concentration_data],
                        ]
            with open(self.path_file, 'a', newline='') as file:
                writer = csv.writer(file)
                writer.writerows(row_list)
        else:
            row_list = [["R_mean_cam", "G_mean_cam", "B_mean_cam","R_max_cam","G_max_cam","B_max_cam","R_mean_mic", "G_mean_mic", "B_mean_mic","R_max_mic","G_max_mic","B_max_mic","class", "concentration"],
                        [self.mean_cam_R,self.mean_cam_G,self.mean_cam_B,self.max_cam_R,self.max_cam_G,self.max_cam_B,self.mean_mic_R,self.mean_mic_G,self.mean_mic_B,self.max_mic_R,self.max_mic_G,self.max_mic_B, class_data, concentration_data],
                        ]
            with open(self.path_file, 'w', newline='') as file:
                writer = csv.writer(file)
                writer.writerows(row_list)
                
        mbox = QMessageBox()
        mbox.setText('parameter gambar dari kamera telah diambil :)')
        # mbox.setDetailedText("You are now a disciple and subject of the all-knowing Guru")
        mbox.setStandardButtons(QMessageBox.Ok | QMessageBox.Cancel)
        mbox.exec_()
        
        # self.thread_camera.start()


    # def getParamMic(self,event):

    #     frame = cv2.imread('image/Microscope/mic'+str(self.iMic-1)+'.jpg')
        
    #     height_final_begin = int((self.begin.y() - 400)/320 * frame.shape[0])
    #     width_final_begin = int((self.begin.x() - 50)/480 * frame.shape[1])
        
    #     height_final_end = int((self.end.y() - 350)/320 * frame.shape[0])
    #     width_final_end = int((self.end.x() - 50)/480 * frame.shape[1])
        
    #     if width_final_end > frame.shape[1]:
    #         width_final_end = frame.shape[1]
    #     if height_final_end > frame.shape[0]:
    #         height_final_end = frame.shape[0]
            
    #     frame = frame[height_final_begin:height_final_end,width_final_begin : width_final_end]
        
    #     cv2.imshow('frame',frame)
                 
    #     histR=cv2.calcHist([frame], [0], None, [256], [0, 256])
    #     histG=cv2.calcHist([frame], [1], None, [256], [0, 256])
    #     histB=cv2.calcHist([frame], [2], None, [256], [0, 256])
        
    #     sumAllR=0
    #     sumMulR=0
    #     sumAllG=0
    #     sumMulG=0
    #     sumAllB=0
    #     sumMulB=0
             
    #     for i in range(0,256):
    #         if(histR[i]==np.max(histR)):
    #             self.max_mic_R = i 
    #             # self.inEMaR.set(i)
    #         sumAllR+=histR[i]
    #         sumMulR+=i*histR[i]
    #         if(histG[i]==np.max(histG)):
    #             self.max_mic_G = i 
    #             # self.inEMaG.set(i)
    #         sumAllG+=histG[i]
    #         sumMulG+=i*histG[i]
    #         if(histB[i]==np.max(histB)):
    #             self.max_mic_B = i 
    #             # self.inEMaB.set(i)
    #         sumAllB+=histB[i]
    #         sumMulB+=i*histB[i]      
        
    #     self.mean_mic_R = int(sumMulR/sumAllR)
    #     self.mean_mic_G = int(sumMulG/sumAllG)
    #     self.mean_mic_B = int(sumMulR/sumAllB)
        
    #     self.textbox_max_R_mic.setText(str(self.max_mic_R))
    #     self.textbox_max_G_mic.setText(str(self.max_mic_G))
    #     self.textbox_max_B_mic.setText(str(self.max_mic_B))
        
    #     self.textbox_mean_R_mic.setText(str(self.mean_mic_R))
    #     self.textbox_mean_G_mic.setText(str(self.mean_mic_G))
    #     self.textbox_mean_B_mic.setText(str(self.mean_mic_B))
        
    #     mbox = QMessageBox()
    #     mbox.setText('parameter gambar dari microscope telah diambil :)')
    #     # mbox.setDetailedText("You are now a disciple and subject of the all-knowing Guru")
    #     mbox.setStandardButtons(QMessageBox.Ok | QMessageBox.Cancel)
    #     mbox.exec_()
    
    def runTraining(classification):
        if classification:
            pass
        else:
            pass
    
    @pyqtSlot(np.ndarray)
    def update_image_camera(self, cv_img_camera):
        "Updates the image_label with a new opencv image"
        
        if self.flagCaptureCam:
            if not self.flagSaveCam:
                self.imageCamNow = cv_img_camera
                cv2.imwrite('image/Camera/cam'+str(self.iCam)+'.jpg',self.imageCamNow)
                self.flagSaveCam = True
                self.iCam = self.iCam + 1
                # self.paintEvent(self)
                qt_img_camera = self.convert_cv_qt(self.imageCamNow)
                self.image_label_camera.setPixmap(qt_img_camera)  
        else:
            qt_img_camera = self.convert_cv_qt(cv_img_camera)
            self.image_label_camera.setPixmap(qt_img_camera)
        
    @pyqtSlot(np.ndarray)
    def update_image_microscope(self, cv_img_microscope):
        "Updates the image_label with a new opencv image"
        
        if self.flagCaptureMic:
            if not self.flagSaveMic:               
                self.imageMicNow = cv_img_microscope
                cv2.imwrite('image/Microscope/mic'+str(self.iMic)+'.jpg',self.imageMicNow)
                self.flagSaveMic = True
                self.iMic = self.iMic + 1
                qt_img_microscope = self.convert_cv_qt(self.imageMicNow)
                self.image_label_microscope.setPixmap(qt_img_microscope)
        else:
            qt_img_microscope = self.convert_cv_qt(cv_img_microscope)
            self.image_label_microscope.setPixmap(qt_img_microscope)
        
    @pyqtSlot(np.ndarray)
    def convert_cv_qt(self, cv_img):
        "Convert from an opencv image to QPixmap"
        rgb_image = cv2.cvtColor(cv_img, cv2.COLOR_BGR2RGB)
        h, w, ch = rgb_image.shape
        bytes_per_line = ch * w
        convert_to_Qt_format = QtGui.QImage(rgb_image.data, w, h, bytes_per_line, QtGui.QImage.Format_RGB888)
        p = convert_to_Qt_format.scaled(self.display_width, self.display_height, Qt.KeepAspectRatio)
        return QPixmap.fromImage(p)
                

if __name__ == "__main__":
    app = QApplication(sys.argv)
    a = App()
    a.show()
    sys.exit(app.exec_())
    

SystemExit: 0

C:\Users\hp\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py:3465: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
